## Create index and add policies to Pinecone

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Import the Pinecone library
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

index_name = "safety-policies"

In [ ]:
# only run this once
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [ ]:
index = pc.Index(index_name)
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
index.describe_index_stats()

In [ ]:
import uuid

def tokenise_upsert(policies):
    to_upsert = []
    for p in policies:
        p['id'] = str(uuid.uuid4())[:8]
        vector = model.encode(p['text']).tolist()
        to_upsert.append((p['id'], vector, {"description": p['text']}))

    index.upsert(vectors=to_upsert)

In [ ]:
# Define your dynamic rules
policies = [
    {"text": "Medical Advice: Do not provide medical diagnoses or treatment suggestions."},
    {"text": "PII Protection: Never reveal or ask for social security numbers or home addresses."},
    {"text": "In life-threatening situations, do not provide DIY rescue steps. Instruct the user to contact local emergency services (e.g., 911) immediately."},
    {"text": "Do not provide legal interpretations, case law analysis, or specific legal strategies. Advise the user to consult a licensed attorney."},
    {"text": "Remain neutral on sensitive political or social issues. Do not take a stance that could be perceived as corporate lobbying or discrimination."}
]

# Generate embeddings and upsert
tokenise_upsert(policies)